# 영화 리뷰 분류: 이진 분류 예제

- 리뷰 텍스트를 기반으로 영화 리뷰를 긍정과 부정로 분류하는 예제

## IMDB 데이터셋
- 케라스 데이터 셋  
- 인터넷 영화 데이터베이스(Internet Movie Database)로부터 가져온 
- 양극단의 리뷰 50,000개로 이루어진 IMDB 데이터셋을 사용


- 훈련 데이터 25,000개와 테스트 데이터 25,000개로 나뉘어 있고 
- 각각 50%는 부정, 50%는 긍정 리뷰로 구성되어 있음

- 전처리되어 있음 : 각 리뷰(단어 시퀀스)가 숫자 시퀀스로 변환되어 있음
- 각 숫자는 사전에 있는 고유한 단어를 나타냄


어휘 사전 (사전)
- 모든 단어에 고유한 번호 부여
- 텍스트 데이터를 다룰 때 기본적으로 수행하는 전처리 과정

### 영화 리뷰 분류 과정
- (1) 데이터셋 준비
- (2) 모델링 

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import numpy as np

### 데이터 준비

In [3]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

train_data / test_data : 리뷰 목록
- 각 리뷰는 단어 인덱스의 리스트
- 단어 시퀀스가 인코딩된 것

train_labels / test_labels (레이블 값)
- 0 : 부정
- 1: 긍정

In [4]:
train_data[:2]
train_labels[:10]

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [5]:
max([max(x) for x in train_data])

9999

#### 인덱스와 단어 확인
- 리뷰 데이터 하나를 원래 영어 단어로 디코딩

In [6]:
word_index = imdb.get_word_index()
# key가 단어, value가 숫자이다

In [7]:
# 구조 변경
reversed_word_index = dict([(val, key) for key, val in word_index.items()])

In [8]:
# 확인
# 참고로 0, 1, 2 인덱스는 리뷰의 시작 특성을 나타내기에 3을 뺀 값을 인덱스로 한다
decoded_review = ' '.join([reversed_word_index.get(i-3, '<???>') for i in train_data[0]])
decoded_review

"<???> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <???> is an amazing actor and now the same being director <???> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <???> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <???> to the two little boy's that played the <???> of norman and paul they were just brilliant children are often left out of the <???> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what

### 데이터 전처리

- 신경망에 숫자 리스트를 입력으로 사용할 수 없기 때문에
- 리스트를 텐서로 변경 : 벡터로 변환 [0, 1, 1, 0, 1, ... 0, 1]
- 멀티-핫 인코딩 방법 사용해서 변환

In [9]:
# a_test = np.array([[1, 2], [3, 4], [5, 6, 7]]) 차원수가 안맞아서 오류
a_test = np.array([[1, 4], [2, 7], [4, 5, 8]], dtype=object)

In [10]:
# a_test을 인코딩한 값을 넣을 배열 미리 생성
result = np.zeros((len(a_test), 9))

In [11]:
for i, d in enumerate(a_test):
    result[i, d] += 1
result

array([[0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 1.]])

#### 패딩 (Padding)
- 문장의 길이를 나타내는 배열의 크기를 동일하게 맞추는 작업  
- 길이가 전부 동일한 문서들에 대하여 하나의 행렬로 인식하고  
- 한꺼번에 묶어서 처리  
- 각 문자의(문서)의 길이가 서로 다른 경우  
- 동일한 길이로 맞추는 빈 부분은 0을 채움  
- 예 : [1, 2]와 [1, 3, 4] 길이가 다른 배열을 길이 5로 동일하게 만들기   
    - [1, 1, 0, 0, 0]  
    - [1, 0, 1, 1, 0]  

In [12]:
def vectorize(sequences, dim=10000):
    arr = np.zeros((len(sequences), dim))
    for idx, seq in enumerate(sequences):
        arr[idx, seq] += 1
    return arr

In [13]:
X_train = vectorize(train_data)
X_test = vectorize(test_data)

In [14]:
y_train = train_labels
y_test = test_labels

In [15]:
X_train[:10]

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [16]:
len(X_train)

25000

In [17]:
# 학습 검증용을 따로 준비
X_val = X_train[:20000]
X_tr = X_train[20000:]
y_val = y_train[:20000]
y_tr = y_train[20000:]

### (2) 모델링 

- (1) 모델 설정  
- (2) 모델 컴파일  
- (3) 모델 학습(실행)  
- (4) 평가 결과 출력 

In [18]:
from keras.models import Sequential
from keras.layers import Dense

In [19]:
# 모델 생성
model_imdb = Sequential()

model_imdb.add(Dense(units=100, input_dim=10000, activation='linear'))
model_imdb.add(Dense(units=10, activation='relu'))
model_imdb.add(Dense(units=1, activation='sigmoid'))

# 컴파일
model_imdb.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

2023-12-04 13:41:12.190848: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-12-04 13:41:12.190899: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-12-04 13:41:12.190908: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-12-04 13:41:12.190967: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-04 13:41:12.190984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
model_imdb.fit(x=X_val, y=y_val, epochs=3, batch_size=4, validation_data=(X_tr, y_tr))

Epoch 1/3
   1/5000 [..............................] - ETA: 27:52 - loss: 0.6957 - accuracy: 0.5000

2023-12-04 13:41:14.016973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


5000/5000 [==============================] - 36s 7ms/step - loss: 0.3483 - accuracy: 0.8545 - val_loss: 0.3152 - val_accuracy: 0.8740
Epoch 2/3
5000/5000 [==============================] - 36s 7ms/step - loss: 0.2720 - accuracy: 0.8951 - val_loss: 0.3642 - val_accuracy: 0.8592
Epoch 3/3
5000/5000 [==============================] - 36s 7ms/step - loss: 0.2124 - accuracy: 0.9213 - val_loss: 0.3689 - val_accuracy: 0.8640


### 학습 및 검증 데이터에 대한 손실과 정확도 시각화

In [21]:
import matplotlib.pyplot as plt

### 모델 저장 및 불러오기 

### 훈련된 모델로 새로운 데이터에 대해 예측하기

모델을 훈련시킨 후에 이를 실전 환경에서 사용 
- predict() 사용
